# The scripts below are for wrangling data within the individual spreadsheets for each LDC for years 1999 and 2001.

Create a new variable "emne" that is True if name of country isn't found in list of developed economies.
List of Developed Economies taken from UNCTAD LDC Report 2004 found here: https://unctad.org/system/files/official-document/ldc2004_en.pdf

In [1]:
import os
import pandas as pd

# Load the list of developed economies from the Excel file
developed_economies_file = "./data/supplementary_data/developed_economies_list.xlsx"
developed_economies_df = pd.read_excel(developed_economies_file, sheet_name="Sheet1")
developed_economies = set(developed_economies_df["country_name"].tolist())


# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path)
    df["emne"] = df["home_economy"].apply(
        lambda x: False if x in developed_economies else True
    )
    df.to_csv(file_path, index=False)


# Directories to process
directories = ["./data/country_spreadsheets/1999", "./data/country_spreadsheets/2001"]

# Process each CSV file in the directories
for directory in directories:
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            process_csv(file_path)

print("Processing complete.")

Processing complete.


Create new variable "nat_res_ind" based on ./data/supplementary_data/bordering_countries.csv

In [2]:
import pandas as pd
import os

# Define the paths to the directories containing the CSV files
dir1 = "./data/country_spreadsheets/1999"  # Update this path as needed
dir2 = "./data/country_spreadsheets/2001"  # Update this path as needed

# Read the natural resource industries into a set with boolean conversion
nat_res_ind_path = "./data/supplementary_data/nat_resource_industries.csv"
nat_res_ind_df = pd.read_csv(nat_res_ind_path)
nat_res_ind_df["nat_resource"] = (
    nat_res_ind_df["nat_resource"].astype(str).str.lower() == "true"
)
nat_res_industries = set(
    nat_res_ind_df[nat_res_ind_df["nat_resource"]]["industry"].str.strip()
)


# Function to process CSV files with filtering for nat_resource == True
def process_csv_files(directory, nat_res_industries):
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            df["industry"] = df["industry"].astype(str).str.strip()
            df["nat_res_ind"] = df["industry"].apply(lambda x: x in nat_res_industries)
            df.to_csv(file_path, index=False)


# Process the directories with the correct reference to nat_res_industries
process_csv_files(dir1, nat_res_industries)
process_csv_files(dir2, nat_res_industries)


Create new variable former_col_power based on coding in ./data/supplementary_data/colonial_rulers_list.csv

In [3]:
import os
import pandas as pd

# Define the paths to the directories containing the CSV files
dir1999 = "./data/country_spreadsheets/1999"  # Replace with the actual path to the 1999 directory
dir2001 = "./data/country_spreadsheets/2001"  # Replace with the actual path to the 2001 directory

# Read the colonial rulers list into a dictionary
colonial_rulers_path = "./data/supplementary_data/colonial_rulers_list.csv"
colonial_rulers_df = pd.read_csv(colonial_rulers_path)
colonial_rulers_dict = colonial_rulers_df.set_index("country")[
    "colonial_rulers"
].to_dict()


# Function to process CSV files and add former_col_power column
def process_csv_files(directory, colonial_rulers_dict):
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            # Extract the country name by removing the last underscore and year
            country = "_".join(filename.split("_")[:-1])
            former_colonial_power = colonial_rulers_dict.get(country, None)
            if former_colonial_power:
                file_path = os.path.join(directory, filename)
                df = pd.read_csv(file_path)
                df["former_col_power"] = df["home_economy"].apply(
                    lambda x: x == former_colonial_power
                )
                df.to_csv(file_path, index=False)


# Process the directories
process_csv_files(dir1999, colonial_rulers_dict)
process_csv_files(dir2001, colonial_rulers_dict)

print("Processed files and added former_col_power column.")

Processed files and added former_col_power column.


Create new variable neighbouring_country for based on coding in neighbouring_country spreadsheet in ./data/supplementary_data/bordering_countries.csv

In [10]:
import os
import pandas as pd

# Define the paths to the directories containing the CSV files
dir1999 = "./data/country_spreadsheets/1999"  # Replace with the actual path to the 1999 directory
dir2001 = "./data/country_spreadsheets/2001"  # Replace with the actual path to the 2001 directory

# Read the bordering countries list into a dictionary
bordering_countries_path = "./data/supplementary_data/bordering_countries.csv"
bordering_countries_df = pd.read_csv(bordering_countries_path)

# Create a dictionary where the key is the country and the value is a list of bordering countries
bordering_countries_dict = {}
for _, row in bordering_countries_df.iterrows():
    country = row["country"]
    borders = row["bordering_countries"]
    if isinstance(borders, str):  # Ensure borders is a string
        bordering_countries_dict[country] = [
            border.replace(" ", "_").lower() for border in borders.split(", ")
        ]

# Function to convert Title Case to snake_case
def to_snake_case(s):
    return s.lower().replace(" ", "_")

# Function to process CSV files and add bordering_country column
def process_csv_files(directory, bordering_countries_dict):
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            # Extract the country name by removing the last underscore and year
            country = "_".join(filename.split("_")[:-1])
            bordering_countries = bordering_countries_dict.get(country, [])


            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            df["bordering_country"] = df["home_economy"].apply(
                lambda x: to_snake_case(x) in bordering_countries
            )

            df.to_csv(file_path, index=False)

# Process the directories
process_csv_files(dir1999, bordering_countries_dict)
process_csv_files(dir2001, bordering_countries_dict)

print("Processed files and added bordering_country column.")

Processed files and added bordering_country column.


### Working with master spreadsheets for 1999 and 2001

Create master spreadsheet for 1999

In [5]:
import os
import pandas as pd

# Directory where the CSV files are located
directory = "./data/country_spreadsheets/1999"  # Change this to your directory path

# List of all CSV files in the directory
csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

# List to store the results
results = []
missing_former_col_power_files = []


# Function to calculate the proportions
def calculate_proportions(df, col):
    total_count = len(df)
    if total_count == 0:
        return None
    mne_count = df[df["emne"] == True].shape[0]
    return mne_count / total_count


# Process each CSV file
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)

    # Extract the country name by splitting the filename at the last underscore
    country = "_".join(file.split("_")[:-1])

    # Calculate the required proportions
    prop_emnes = calculate_proportions(df, "emne")

    if "nat_res_ind" in df.columns:
        prop_emnes_excl_nat_res = calculate_proportions(
            df[df["nat_res_ind"] == False], "emne"
        )
    else:
        prop_emnes_excl_nat_res = prop_emnes

    if "former_col_power" in df.columns:
        prop_emnes_excl_former_col_power = calculate_proportions(
            df[df["former_col_power"] == False], "emne"
        )
    else:
        prop_emnes_excl_former_col_power = prop_emnes
        missing_former_col_power_files.append(file)

    # Check if there are zero companies
    zero_mnes = len(df) == 0

    # Check if there are zero companies excluding natural resource companies
    zero_mnes_excl_nat_res = (
        len(df[df["nat_res_ind"] == False]) == 0
        if "nat_res_ind" in df.columns
        else zero_mnes
    )

    # Check if there are zero companies excluding former colonial powers
    zero_mnes_excl_former_col_power = (
        len(df[df["former_col_power"] == False]) == 0
        if "former_col_power" in df.columns
        else zero_mnes
    )


    # Append the results
    results.append(
        {
            "country": country,
            "prop_emnes": prop_emnes,
            "prop_emnes_excl_nat_res": prop_emnes_excl_nat_res,
            "prop_emnes_excl_former_col_power": prop_emnes_excl_former_col_power,
            "zero_mnes": zero_mnes,
            "zero_mnes_excl_nat_res": zero_mnes_excl_nat_res,
            "zero_mnes_excl_former_col_power": zero_mnes_excl_former_col_power,
        }
    )

# Create a DataFrame from the results
master_df = pd.DataFrame(results)

# Save the master DataFrame to a CSV file
output_path = os.path.join(directory, "ldcs_1999_master.csv")
master_df.to_csv(output_path, index=False)

print(f"Master CSV file created at: {output_path}")

# Print the list of files missing the former_col_power column
if missing_former_col_power_files:
    print("The following files were missing the 'former_col_power' column:")
    for missing_file in missing_former_col_power_files:
        print(missing_file)
else:
    print("No files were missing the 'former_col_power' column.")

Master CSV file created at: ./data/country_spreadsheets/1999/ldcs_1999_master.csv
No files were missing the 'former_col_power' column.


Create master spreadsheet for 2001

In [7]:
import os
import pandas as pd

# Directory where the CSV files are located
directory = "./data/country_spreadsheets/2001"  # Change this to your directory path

# List of all CSV files in the directory
csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

# List to store the results
results = []
missing_former_col_power_files = []


# Function to calculate the proportions
def calculate_proportions(df, col):
    total_count = len(df)
    if total_count == 0:
        return None
    mne_count = df[df["emne"] == True].shape[0]
    return mne_count / total_count


# Process each CSV file
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)

    # Extract the country name by splitting the filename at the last underscore
    country = "_".join(file.split("_")[:-1])

    # Calculate the required proportions
    prop_emnes = calculate_proportions(df, "emne")

    if "nat_res_ind" in df.columns:
        prop_emnes_excl_nat_res = calculate_proportions(
            df[df["nat_res_ind"] == False], "emne"
        )
    else:
        prop_emnes_excl_nat_res = prop_emnes

    if "former_col_power" in df.columns:
        prop_emnes_excl_former_col_power = calculate_proportions(
            df[df["former_col_power"] == False], "emne"
        )
    else:
        prop_emnes_excl_former_col_power = prop_emnes
        missing_former_col_power_files.append(file)

    # Check if there are zero companies
    zero_mnes = len(df) == 0

    # Check if there are zero companies excluding natural resource companies
    zero_mnes_excl_nat_res = (
        len(df[df["nat_res_ind"] == False]) == 0
        if "nat_res_ind" in df.columns
        else zero_mnes
    )

    # Check if there are zero companies excluding former colonial powers
    zero_mnes_excl_former_col_power = (
        len(df[df["former_col_power"] == False]) == 0
        if "former_col_power" in df.columns
        else zero_mnes
    )

    # Append the results
    results.append(
        {
            "country": country,
            "prop_emnes": prop_emnes,
            "prop_emnes_excl_nat_res": prop_emnes_excl_nat_res,
            "prop_emnes_excl_former_col_power": prop_emnes_excl_former_col_power,
            "zero_mnes": zero_mnes,
            "zero_mnes_excl_nat_res": zero_mnes_excl_nat_res,
            "zero_mnes_excl_former_col_power": zero_mnes_excl_former_col_power,
        }
    )

# Create a DataFrame from the results
master_df = pd.DataFrame(results)

# Save the master DataFrame to a CSV file
output_path = os.path.join(directory, "ldcs_2001_master.csv")
master_df.to_csv(output_path, index=False)

print(f"Master CSV file created at: {output_path}")

# Print the list of files missing the former_col_power column
if missing_former_col_power_files:
    print("The following files were missing the 'former_col_power' column:")
    for missing_file in missing_former_col_power_files:
        print(missing_file)
else:
    print("No files were missing the 'former_col_power' column.")

Master CSV file created at: ./data/country_spreadsheets/2001/ldcs_2001_master.csv
No files were missing the 'former_col_power' column.
